## A tutorial on the xmask package

### How to install

Get the latest version of xsuite, cpymad, etc:

    pip install xsuite
    pip install cpymad

Get hllhc15 
    
    git clone https://github.com/lhcopt/hllhc15.git

Uninstall the old version of xmask (not compatible with lhc errors as for now):

    pip uninstall xmask

Install the new version of xmask locally, including submodules for lhc errors:

    git clone https://github.com/xsuite/xmask.git xmask_github
    cd xmask_github
    git submodule init
    git submodule update
    pip install -e .

One can precompile the code in xsuite for faster execution:

    xsuite-compile


### Imports

In [1]:
# Standard imports
import os
import pandas as pd
import matplotlib.pyplot as plt

# Xmask
import xmask as xm
import xtrack as xt

# Cpymad (Python wrapper for MAD-X, used to build sequence and apply optics)
from cpymad.madx import Madx

# Import user-defined optics-specific tools (functions to build sequence and apply optics)
from tools import optics_specific_tools_hlhc15 as ost

# Plotting function
from tools.plotting_functions import plot_all

# Plotly renderer
import plotly.io as pio

# ! Renderer should be set to 'notebook' or 'vscode' (depending on the IDE used) for interactive plots
pio.renderers.default = "svg"


In [2]:
# Constants to compartimentalize the code
DO_STEP_0 = True
DO_EXPLORATION_STEP_0 = True
DO_STEP_1 = True
DO_EXPLORATION_STEP_1 = True
DO_STEP_2 = True
DO_EXPLORATION_STEP_2 = True
DO_STEP_3 = True
DO_EXPLORATION_STEP_3 = True
DO_STEP_4 = True


### Build collider from MAD model

In [3]:
config_mad_model = None
if DO_STEP_0:
    # Get mad config from general config file
    with open("config.yaml", "r") as fid:
        config = xm.yaml.load(fid)
    config_mad_model = config["config_mad"]

config_mad_model


{'links': {'acc-models-lhc': './hllhc15'},
 'optics_file': 'acc-models-lhc/flatcc/opt_flatvh_75_180_1500_thin.madx',
 'beam_config': {'lhcb1': {'beam_energy_tot': 7000},
  'lhcb2': {'beam_energy_tot': 7000}},
 'enable_imperfections': False,
 'enable_knob_synthesis': False,
 'ver_hllhc_optics': 1.5,
 'ver_lhc_run': None,
 'pars_for_imperfections': {'par_myseed': 1,
  'par_correct_for_D2': 0,
  'par_correct_for_MCBX': 0,
  'par_on_errors_LHC': 1,
  'par_off_errors_Q4_inIP15': 0,
  'par_off_errors_Q5_inIP15': 0,
  'par_on_errors_MBH': 1,
  'par_on_errors_Q4': 1,
  'par_on_errors_D2': 1,
  'par_on_errors_D1': 1,
  'par_on_errors_IT': 1,
  'par_on_errors_MCBRD': 0,
  'par_on_errors_MCBXF': 0,
  'par_on_errors_NLC': 0,
  'par_write_errortable': 1}}

In [4]:
if DO_STEP_0:
    # Make mad environment
    xm.make_mad_environment(links=config_mad_model["links"])

    # Start mad
    mad_b1b2 = Madx(command_log="mad_collider.log")
    mad_b4 = Madx(command_log="mad_b4.log")


rm: cannot remove 'acc-models-lhc': No such file or directory
rm: cannot remove 'temp': No such file or directory



  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.08.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2022.02.25               +
  + Execution date: 2023.04.04 13:42:01      +
  ++++++++++++++++++++++++++++++++++++++++++++

  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.08.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2022.02.25               +
  + Execution date: 2023.04.04 13:42:01      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [5]:
if DO_STEP_0:
    # Build sequences for beam 1 and 4
    ost.build_sequence(mad_b1b2, mylhcbeam=1)
    ost.build_sequence(mad_b4, mylhcbeam=4)


In [ ]:
if DO_STEP_0:
    # Apply optics (only for b1b2, b4 will be generated from b1b2)
    ost.apply_optics(mad_b1b2, optics_file=config_mad_model["optics_file"])


In [ ]:
if DO_STEP_0:
    # Build xsuite collider
    collider = xm.lhc.build_xsuite_collider(
        sequence_b1=mad_b1b2.sequence.lhcb1,
        sequence_b2=mad_b1b2.sequence.lhcb2,
        sequence_b4=mad_b4.sequence.lhcb2,
        beam_config=config_mad_model["beam_config"],
        enable_imperfections=config_mad_model["enable_imperfections"],
        enable_knob_synthesis=config_mad_model["enable_knob_synthesis"],
        pars_for_imperfections=config_mad_model["pars_for_imperfections"],
        ver_lhc_run=config_mad_model["ver_lhc_run"],
        ver_hllhc_optics=config_mad_model["ver_hllhc_optics"],
    )


In [ ]:
if DO_STEP_0:
    # Create output folder if it does not exist
    if not os.path.exists("output"):
        os.makedirs("output")

    # Save to file
    collider.to_json("output/collider_00_from_mad.json")


In [ ]:
if DO_STEP_0:
    # Remove all the temporaty files created in the process of building collider
    os.remove("mad_collider.log")
    os.remove("mad_b4.log")
    os.rmdir("temp")
    os.unlink("errors")
    os.unlink("acc-models-lhc")


### Explore vanilla collider with no knobs nor bb

In [ ]:
if DO_EXPLORATION_STEP_0:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_00_from_mad.json")

    # Build trackers to be able to twiss, track, etc
    collider.build_trackers()


In [ ]:
if DO_EXPLORATION_STEP_0:
    # A collider contain a list of lines: beam 1 and beam 2, and beam 1 and beam 2 for the closed orbit (all knobs set to zero)
    print(collider.lines.keys())


In [ ]:
if DO_EXPLORATION_STEP_0:

    # A collider also contain some knobs
    print(list(collider.vars._get_value().items())[:10])

    # And same for the line inside of the collider
    print(list(collider.lhcb1.vars._get_value().items())[:10])


In [ ]:
if DO_EXPLORATION_STEP_0:
    # Knobs values can be accessed and modified indivually
    print("Value before modification: ", collider.vars["on_x5"]._get_value())
    collider.vars["on_x5"] = 100
    print("Value after modification: ", collider.vars["on_x5"]._get_value())
    collider.vars["on_x5"] = 0


In [ ]:
tw1 = None
if DO_EXPLORATION_STEP_0:
    # One can twiss the lines in the collider (in 4D, since RF hasn't been added yet)
    # Note that the twiss in the second line is reversed, to get the same s coordinates for both lines
    tw1_step0 = collider.lhcb1.twiss(method="4d")
    tw2_step0 = collider.lhcb2.twiss(method="4d").reverse()
    tw1_co_step0 = collider.lhcb1_co_ref.twiss(method="4d")
    tw2_co_step0 = collider.lhcb2_co_ref.twiss(method="4d").reverse()
    print(tw1_step0.to_pandas())


In [ ]:
if DO_EXPLORATION_STEP_0:
    # TwissTable offers fancy possibilites for indexing
    print(
        tw1_step0.rows[
            "ip4":"ip6",
            "mq.*",
            3500:3600:"s",
            tw1_step0.betx > 500,
            ["mqy.6r4.b1..1", "mqy.6r4.b1..2"],
        ].cols["s", "betx"]
    )


In [ ]:
if DO_EXPLORATION_STEP_0:
    # We can check what the no beambeam elements have been added, e.g. in ip5
    tw1_step0 = collider.lhcb1.twiss(method="4d")
    print(tw1_step0.rows["ip4":"ip6", "bb.*"].cols["s"])


In [ ]:
if DO_EXPLORATION_STEP_0:
    # Check that for now the twiss is the same for the standard line and the reference line (since knobs haven't been applied)
    tw1_step0_pd = tw1_step0.to_pandas().drop(columns=["W_matrix"])
    tw1_co_step0_pd = tw1_co_step0.to_pandas().drop(columns=["W_matrix"])
    df_diff = pd.concat([tw1_step0_pd, tw1_co_step0_pd]).drop_duplicates(keep=False)
    print(df_diff)


In [ ]:
p = None
if DO_EXPLORATION_STEP_0:
    # For now, the beams are equal to their reference (since knobs haven't been applied)
    p = plot_all(tw1_step0_pd, tw1_co_step0_pd, "beam 1", "beam 1 co ref")
p


In [ ]:
if DO_EXPLORATION_STEP_0:
    # One can also get the corresponding surveys
    sv1_step0 = collider.lhcb1.survey()
    sv2_step0 = collider.lhcb2.survey().reverse()
    sv1_co_step0 = collider.lhcb1_co_ref.survey()
    sv2_co_step0 = collider.lhcb2_co_ref.survey().reverse()

    print(sv1_step0.to_pandas())


In [ ]:
p = None
if DO_EXPLORATION_STEP_0:
    # We can check what the beams looks like along the collider
    tw1_step0_pd = tw1_step0.to_pandas()
    tw2_step0_pd = tw2_step0.to_pandas()
    p = plot_all(tw1_step0_pd, tw2_step0_pd, "beam 1", "beam 2")
p


### Set up beambeam lenses (they stay inactive and not configured) 

In [ ]:
config_bb = None
if DO_STEP_1:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_00_from_mad.json")

    # Get beambeam config from general config file
    with open("config.yaml", "r") as fid:
        config = xm.yaml.load(fid)
    config_bb = config["config_beambeam"]

config_bb


In [ ]:
if DO_STEP_1:
    # Install beam-beam lenses (inactive and not configured)
    collider.install_beambeam_interactions(
        clockwise_line="lhcb1",
        anticlockwise_line="lhcb2",
        ip_names=["ip1", "ip2", "ip5", "ip8"],
        num_long_range_encounters_per_side=config_bb["num_long_range_encounters_per_side"],
        num_slices_head_on=config_bb["num_slices_head_on"],
        harmonic_number=35640,
        bunch_spacing_buckets=config_bb["bunch_spacing_buckets"],
        sigmaz=config_bb["sigma_z"],
    )


In [ ]:
if DO_STEP_1:
    # Save collider as json
    collider.to_json("output/collider_01_bb_off.json")


### Explore the collider after setting up the beambeam lenses

In [ ]:
if DO_EXPLORATION_STEP_1:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_01_bb_off.json")

    # Build trackers to be able to twiss, track, etc
    collider.build_trackers()


In [ ]:
if DO_EXPLORATION_STEP_1:
    # We can check what the beams elements have been added
    tw1_step1 = collider.lhcb1.twiss(method="4d")
    print(tw1_step1.rows["ip4":"ip5", "bb.*"].cols["s"])


In [ ]:
if DO_EXPLORATION_STEP_1:
    # The beam is not anymore equal to its reference, as the reference has no beam-beam elements, which introduce many drifts
    tw1_step1_pd = tw1_step1.to_pandas().drop(columns=["W_matrix"]).round(4)
    tw1_co_step1 = collider.lhcb1_co_ref.twiss(method="4d")
    tw1_co_step1_pd = tw1_co_step1.to_pandas().drop(columns=["W_matrix"]).round(4)
    df_diff = pd.concat([tw1_step1_pd, tw1_co_step1_pd]).drop_duplicates(keep=False)
    print(df_diff)


In [ ]:
p = None
if DO_EXPLORATION_STEP_1:
    # However, the observables are still identical
    p = plot_all(tw1_step1_pd, tw1_co_step1_pd, "beam 1", "beam 1 co ref")
p


### Configure knobs and tuning

In [ ]:
config_knobs_and_tuning = None
if DO_STEP_2:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_01_bb_off.json")
    collider.build_trackers()

    # Get beambeam config from general config file
    with open("config.yaml", "r") as fid:
        config = xm.yaml.load(fid)
    config_knobs_and_tuning = config["config_knobs_and_tuning"]

config_knobs_and_tuning


In [ ]:
if DO_STEP_2:
    # Set all knobs (crossing angles, dispersion correction, rf, crab cavities,
    # experimental magnets, etc.)
    for kk, vv in config_knobs_and_tuning["knob_settings"].items():
        collider.vars[kk] = vv


In [ ]:
if DO_STEP_2:
    # Run the script to generate configuration for orbit correction
    %run 'tools/gen_config_orbit_correction.py'

    # Adjust tune and chromaticity
    for line_name in ['lhcb1', 'lhcb2']:

        knob_names = config_knobs_and_tuning['knob_names'][line_name]

        targets = {
            'qx': config_knobs_and_tuning['qx'][line_name],
            'qy': config_knobs_and_tuning['qy'][line_name],
            'dqx': config_knobs_and_tuning['dqx'][line_name],
            'dqy': config_knobs_and_tuning['dqy'][line_name],
        }

        xm.machine_tuning(line=collider[line_name],
            enable_closed_orbit_correction=True,
            enable_linear_coupling_correction=True,
            enable_tune_correction=True,
            enable_chromaticity_correction=True,
            knob_names=knob_names,
            targets=targets,
            line_co_ref=collider[line_name+'_co_ref'],
            co_corr_config=config_knobs_and_tuning['closed_orbit_correction'][line_name])
        
    # Remove temporary files
    os.remove('corr_co_lhcb1.json')
    os.remove('corr_co_lhcb2.json')

In [ ]:
if DO_STEP_2:
    # Save collider as json
    collider.to_json("output/collider_02_tuned_bb_off.json")


### Explore the collider with the new tune

In [ ]:
if DO_EXPLORATION_STEP_2:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_02_tuned_bb_off.json")

    # Build trackers to be able to twiss, track, etc
    collider.build_trackers()


In [ ]:
if DO_EXPLORATION_STEP_2:
    # Compute new twiss
    tw1_step2 = collider.lhcb1.twiss(method="4d")
    tw2_step2 = collider.lhcb2.twiss(method="4d").reverse()
    tw1_co_step2 = collider.lhcb1_co_ref.twiss(method="4d")
    tw2_co_step2 = collider.lhcb2_co_ref.twiss(method="4d").reverse()


In [ ]:
p = None

if DO_EXPLORATION_STEP_2:
    # Check how the crossing angle has changed
    tw1_step2_pd = tw1_step2.to_pandas()
    tw1_co_step2_pd = tw1_co_step2.to_pandas()

    p = plot_all(tw1_step1_pd, tw1_step2_pd, "beam 1 before tuning ", "beam 1")

p


In [ ]:
p = None



if DO_EXPLORATION_STEP_2:
    # Check how the crossing angle has changed
    tw1_step2_pd = tw1_step2.to_pandas()
    tw1_co_step2_pd = tw1_co_step2.to_pandas()

    p = plot_all(tw1_step2_pd, tw1_co_step2_pd, "beam 1", "beam 1 reference closed orbit")
p


### Set beambeam interactions

In [ ]:
config_bb = None
if DO_STEP_3:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_02_tuned_bb_off.json")
    collider.build_trackers()

    # Get beambeam config from general config file
    with open("config.yaml", "r") as fid:
        config = xm.yaml.load(fid)
    config_bb = config["config_beambeam"]

config_bb


In [ ]:
if DO_STEP_3:
    # Configure beam-beam lenses
    collider.configure_beambeam_interactions(
        num_particles=config_bb["num_particles_per_bunch"],
        nemitt_x=config_bb["nemitt_x"],
        nemitt_y=config_bb["nemitt_y"],
    )


In [ ]:
if DO_STEP_3:
    # Save collider as json
    collider.to_json("output/collider_03_tuned_bb_on.json")


### Explore final state of the collider

In [ ]:
if DO_EXPLORATION_STEP_3:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_03_tuned_bb_on.json")

    # Build trackers to be able to twiss, track, etc
    collider.build_trackers()


In [ ]:
if DO_EXPLORATION_STEP_3:
    # Compute new twiss
    tw1_step3 = collider.lhcb1.twiss(method="4d")
    tw2_step3 = collider.lhcb2.twiss(method="4d").reverse()
    tw1_co_step3 = collider.lhcb1_co_ref.twiss(method="4d")
    tw2_co_step3 = collider.lhcb2_co_ref.twiss(method="4d").reverse()


In [ ]:
p = None
if DO_EXPLORATION_STEP_3:
    # Check how the crossing angle has changed
    tw1_step3_pd = tw1_step3.to_pandas()
    tw1_co_step3_pd = tw1_co_step3.to_pandas()

    p = plot_all(tw1_step2_pd, tw1_step3_pd, "beam 1 before setting bb ", "beam 1")

p


In [ ]:
p = None
if DO_EXPLORATION_STEP_3:
    p = plot_all(tw1_step3_pd, tw1_co_step3_pd, "beam 1", "beam 1 reference closed orbit")
p


### Compute tune footprint

In [ ]:
if DO_STEP_4:
    # Collider can by reloaded from json file
    collider = xt.Multiline.from_json("output/collider_03_tuned_bb_on.json")

    # Build trackers to be able to twiss, track, etc
    collider.build_trackers()


In [ ]:
# Compute and plot footprint
fp0 = collider["lhcb1"].get_footprint(nemitt_x=2.5e-6, nemitt_y=2.5e-6)

fp_polar = collider["lhcb1"].get_footprint(
    nemitt_x=2.5e-6,
    nemitt_y=2.5e-6,
    linear_rescale_on_knobs=[xt.LinearRescale(knob_name="beambeam_scale", v0=0.0, dv=0.1)],
)


fp_ua = collider["lhcb1"].get_footprint(
    nemitt_x=2.5e-6,
    nemitt_y=2.5e-6,
    mode="uniform_action_grid",
    linear_rescale_on_knobs=[xt.LinearRescale(knob_name="beambeam_scale", v0=0.0, dv=0.1)],
)


plt.close("all")

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)
fp0.plot(ax=ax1, label="no rescale bb")
plt.suptitle("Polar mode (default) - no rescale on beambeam")

fig2 = plt.figure(2)
ax2 = fig2.add_subplot(111, sharex=ax1, sharey=ax1)
fp_polar.plot(ax=ax2, label="rescale bb")
plt.suptitle("Polar mode (default) - linear rescale on beambeam")

fig3 = plt.figure(3)
ax3 = fig3.add_subplot(111, sharex=ax1, sharey=ax1)
fp_ua.plot()
plt.suptitle("Uniform action grid mode - linear rescale on beambeam")

plt.show()
